In [290]:
import os
import sys
import numpy as np
import pandas as pd

In [291]:
import matplotlib.pyplot as plt
%matplotlib inline

### 1.Train data : total 120 days information ( 9448 ~ 9567, unit: day ) 
### 2.Test data : future 30 days information (9568 ~ 9597, unit: day ) 
### 3.Train label : transaction record with date

#### First try:
    - 30 days feature (X)   =>  next 30 days target Y{0,1}
    - 4 binary classification models

In [292]:
# TBN_CC_APPLY.csv
CC_df = pd.read_csv('TBN_CC_APPLY.csv')
print(f'the data shape is : {CC_df.shape}')
CC_df.head()

the data shape is : (54393, 2)


,CUST_NO,TXN_DT
0,8H_N_K5ICM4NU7OG,9519
1,HXD9GCY3SPIYLJS0,9451
2,KSBOZ-TBPUDD9NMO,9502
3,MU8ROCNFDGSL0HJC,9549
4,RGZZAUSNHNXXESM4,9561


In [293]:
# Check missing data
CC_df.isnull().sum() / CC_df.isnull().count()

CUST_NO    0.0
TXN_DT     0.0
dtype: float64

In [294]:
# convert to int data type
CC_df['TXN_DT'] = CC_df['TXN_DT'].values.astype(int) 

In [295]:
month_boundary = [9477, 9507, 9537, 9567]

def make_month_data_dict(whole_data=CC_df, 
                         month_boundary=month_boundary, 
                        target_col='TXN_DT',):
    month_data_dict = {}
    
    for idx, day in enumerate(month_boundary):
        month_df = whole_data[whole_data[target_col]<=day]
        other_df = whole_data[whole_data[target_col]>day]
        month_data_dict[idx+1] = month_df
        whole_data = other_df
    return month_data_dict

month_data_dict = make_month_data_dict(whole_data=CC_df, month_boundary=month_boundary)

# Browse Behavior Data

In [296]:
# TBN_CUST_BEHAVIOR.csv : Browse Behavior

cust_browse_df = pd.read_csv('./TBN_CUST_BEHAVIOR.csv')
print(f'The browse data shape : {cust_browse_df.shape}')
cust_browse_df.head()

The browse data shape : (2209864, 3)


,CUST_NO,VISITDATE,PAGE
0,AZTHNWQ_LXMGIMYG,9462,http://www.esunbank.com.tw/gygrt/e2c/iougkjr/
1,AZTHNWQ_LXMGIMYG,9528,https://www.esunbank.com.tw/gygrt/wgdqth/gsxri...
2,3PY428CHUQBULFIG,9458,https://www.esunbank.com.tw/edrn/deoxt/rgws-cg...
3,JVPD1QUJWVLMZU8S,9457,https://www.esunbank.com.tw/edrn/pgusordq/fgpo...
4,JVPD1QUJWVLMZU8S,9485,https://www.esunbank.com.tw/edrn/pgusordq/fgpo...


In [297]:
browse_month_data_dict = make_month_data_dict(whole_data=cust_browse_df, 
                                              month_boundary=month_boundary,
                                              target_col='VISITDATE',)

In [298]:
for key in range(1, 5):
    page_class = []
    
    
    browse_month_data_dict[key] = browse_month_data_dict[key].reset_index(drop=True)

    for idx in range(len(browse_month_data_dict[key])):
        page_class.append( browse_month_data_dict[key]['PAGE'][idx].split('/')[3] )
        
    browse_month_data_dict[key]['page_class'] = page_class
    del browse_month_data_dict[key]['PAGE']
    
    train_dummy = pd.get_dummies(browse_month_data_dict[key]['page_class'])
    Browse_count = np.sum(train_dummy.values, axis=1)
    browse_month_data_dict[key]['Browse_count'] = Browse_count
        
    browse_month_data_dict[key] = browse_month_data_dict[key].drop(columns='page_class')

In [299]:
browse_month_data_dict[1].head()

,CUST_NO,VISITDATE,Browse_count
0,AZTHNWQ_LXMGIMYG,9462,1
1,3PY428CHUQBULFIG,9458,1
2,JVPD1QUJWVLMZU8S,9457,1
3,JVPD1QUJWVLMZU8S,9471,1
4,JVPD1QUJWVLMZU8S,9455,1


# Custom personal data

In [300]:
# TBN_CIF.csv' 

cust_persional_df = pd.read_csv('./TBN_CIF.csv')
print(f'The browse data shape : {cust_persional_df.shape}')
cust_persional_df.head()

The browse data shape : (187679, 8)


,CUST_NO,AGE,CHILDREN_CNT,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,UFUIMEGMK2KTIWI4,1,NaN,2.0,NaN,NaN,NaN,NaN
1,3HJN2X_LGBHBKILI,3,0.0,9393.0,NaN,NaN,1.0,NaN
2,MNEZK2O3OE-JHYBW,4,0.0,2912.0,3.0,M,2.0,2.0
3,W7ADZSWYXEMHB7JQ,4,0.0,8332.0,5.0,M,2.0,1.0
4,ODB6BA6ORHL2UYZE,4,0.0,744.0,3.0,M,2.0,1.0


In [301]:
# Check missing data
cust_persional_df.isnull().sum().sort_values(ascending=False)

WORK_MTHS            36190
EDU_CODE             28988
GENDER_CODE           1994
CHILDREN_CNT           432
INCOME_RANGE_CODE      382
CUST_START_DT            0
AGE                      0
CUST_NO                  0
dtype: int64

In [302]:
del cust_persional_df['CHILDREN_CNT'] # almost zero value

In [303]:
# Fill missing data by 0

# Fill gender missing data by 'M'
cust_persional_df['GENDER_CODE']  = np.array(cust_persional_df['GENDER_CODE'] == 'M').astype(np.int)

#Fill income data by mode (1)
cust_persional_df['INCOME_RANGE_CODE'] = cust_persional_df['INCOME_RANGE_CODE'].fillna(1.)

# Fill other missing data by 0
cust_persional_df = cust_persional_df.fillna(0.)

# Make training data

In [304]:
for idx in range(1,4):
    month_train_X = pd.DataFrame()
    month_train_X['CUST_NO'] = month_data_dict[idx].groupby(['CUST_NO']).sum().index
    month_train_X['count_x'] = list(month_data_dict[idx].groupby(['CUST_NO']).count()['TXN_DT'])
    # Note: Check each index after groupby is same
    
    #Add browse behavior
    browse_train_x = browse_month_data_dict[idx].groupby('CUST_NO').sum()
    month_train_X = month_train_X.merge(browse_train_x, how='outer', left_on='CUST_NO', right_on='CUST_NO') 
    
    #Add cusotm persional data
    month_train_X = month_train_X.merge(cust_persional_df,  how='outer', left_on='CUST_NO', right_on='CUST_NO')
    
    
    
    month_train_Y = month_data_dict[idx+1].groupby(['CUST_NO']).count()
    month_train_Y['TXN_DT']
    
    merge_df = month_train_X.merge(month_train_Y, how='outer', left_on='CUST_NO', right_on='CUST_NO')
    
    if idx == 1:
        train_data = merge_df.copy()
        continue
        
    train_data = pd.concat([train_data, merge_df], axis=0)

In [305]:
print(f'The training data shape : {train_data.shape}')
train_data.head()

The training data shape : (748792, 11)


,CUST_NO,count_x,VISITDATE,Browse_count,AGE,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS,TXN_DT
0,---CHVW7DUN8SZLO,2.0,28416.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,--KEX0HYP72TSBOQ,1.0,NaN,NaN,4.0,9156.0,6.0,0.0,1.0,1.0,NaN
2,--ONGLWHY32XCGQQ,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,--RW6XQMCPW_UQU4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,--W-DIDC-Q99XJHA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
train_data['label'] = np.array(train_data.isna()['TXN_DT'] == False).astype(np.int)
train_data = train_data.fillna(0)
del train_data['TXN_DT']

In [307]:
# Check test Y
test_data = pd.DataFrame()

test_data['CUST_NO'] = pd.read_csv('./TBN_Y_ZERO.csv')['CUST_NO']
print(f'the data shape is : {test_data.shape}')
test_data[:10]

the data shape is : (30000, 1)


,CUST_NO
0,_PT5HFBEZJKOZ934
1,6STXUMWZRDCGSDDU
2,JDVF4U8JUANEID68
3,8I6SQDGP9OQYUN1M
4,R-TRDUV3GHTID31I
5,J0DDOZLDFF03QBKW
6,HWAZJ_IO2-GACG_C
7,TREOAHKDWDKDT3UQ
8,SA1L7XU6FBENQNT8
9,K0RCWGLXIKGPZNCI


In [308]:
month_test_X = pd.DataFrame()
month_test_X['CUST_NO'] = month_data_dict[4].groupby(['CUST_NO']).sum().index
month_test_X['count_x'] = list(month_data_dict[4].groupby(['CUST_NO']).count()['TXN_DT'])


#Add browse behavior
browse_test_x = browse_month_data_dict[4].groupby('CUST_NO').sum()
month_test_X = month_test_X.merge(browse_test_x, how='outer', left_on='CUST_NO', right_on='CUST_NO') 

#Add cusotm persional data
month_test_X = month_test_X.merge(cust_persional_df,  how='outer', left_on='CUST_NO', right_on='CUST_NO')

In [309]:
test_data = test_data.merge(month_test_X, how='left', left_on='CUST_NO', right_on='CUST_NO')
print(f'The test data shape : {test_data.shape}')
test_data = test_data.fillna(0)
test_data.head()

The test data shape : (30000, 10)


,CUST_NO,count_x,VISITDATE,Browse_count,AGE,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,_PT5HFBEZJKOZ934,0.0,57328.0,6.0,4.0,2912.0,4.0,0.0,1.0,1.0
1,6STXUMWZRDCGSDDU,0.0,114708.0,12.0,2.0,7686.0,3.0,1.0,1.0,1.0
2,JDVF4U8JUANEID68,0.0,57329.0,6.0,4.0,2484.0,3.0,0.0,4.0,1.0
3,8I6SQDGP9OQYUN1M,0.0,28659.0,3.0,1.0,7754.0,6.0,0.0,1.0,1.0
4,R-TRDUV3GHTID31I,0.0,28674.0,3.0,4.0,4062.0,2.0,1.0,1.0,1.0


# Write data to .csv file

In [310]:
def make_same_order(train_df, test_df):
    train_X = pd.DataFrame()
    train_y = pd.DataFrame()
    test_X = pd.DataFrame()
    for col in test_df.columns:
        train_X[col] = train_df[col]
        test_X[col] = test_df[col]
        
    train_y['label'] = train_df['label']
    return train_X, train_y, test_X

train_X, train_y, test_X = make_same_order(train_df=train_data, test_df=test_data)

In [311]:
print(train_X.shape)
train_X.head()

(748792, 10)


,CUST_NO,count_x,VISITDATE,Browse_count,AGE,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,---CHVW7DUN8SZLO,2.0,28416.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,--KEX0HYP72TSBOQ,1.0,0.0,0.0,4.0,9156.0,6.0,0.0,1.0,1.0
2,--ONGLWHY32XCGQQ,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,--RW6XQMCPW_UQU4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,--W-DIDC-Q99XJHA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [312]:
print(train_y.shape)
train_y.head()

(748792, 1)


,label
0,0
1,0
2,0
3,0
4,0


In [313]:
print(test_X.shape)
test_X.head()

(30000, 10)


,CUST_NO,count_x,VISITDATE,Browse_count,AGE,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,_PT5HFBEZJKOZ934,0.0,57328.0,6.0,4.0,2912.0,4.0,0.0,1.0,1.0
1,6STXUMWZRDCGSDDU,0.0,114708.0,12.0,2.0,7686.0,3.0,1.0,1.0,1.0
2,JDVF4U8JUANEID68,0.0,57329.0,6.0,4.0,2484.0,3.0,0.0,4.0,1.0
3,8I6SQDGP9OQYUN1M,0.0,28659.0,3.0,1.0,7754.0,6.0,0.0,1.0,1.0
4,R-TRDUV3GHTID31I,0.0,28674.0,3.0,4.0,4062.0,2.0,1.0,1.0,1.0


In [314]:
train_X.to_csv('./pre_data/CC_train_X.csv')
train_y.to_csv('./pre_data/CC_train_y.csv')
test_X.to_csv('./pre_data/CC_test_X.csv')

# Deal with imbalance data

In [319]:
train_X = pd.read_csv('./pre_data/CC_train_X.csv', index_col=0)
train_X.head()

,CUST_NO,count_x,VISITDATE,Browse_count,AGE,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,---CHVW7DUN8SZLO,2.0,28416.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,--KEX0HYP72TSBOQ,1.0,0.0,0.0,4.0,9156.0,6.0,0.0,1.0,1.0
2,--ONGLWHY32XCGQQ,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,--RW6XQMCPW_UQU4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,--W-DIDC-Q99XJHA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [320]:
train_y = pd.read_csv('./pre_data/CC_train_y.csv', index_col=0)
train_y.head()

,label
0,0
1,0
2,0
3,0
4,0


In [321]:
test_X = pd.read_csv('./pre_data/CC_test_X.csv', index_col=0)
test_X.head()

,CUST_NO,count_x,VISITDATE,Browse_count,AGE,CUST_START_DT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,_PT5HFBEZJKOZ934,0.0,57328.0,6.0,4.0,2912.0,4.0,0.0,1.0,1.0
1,6STXUMWZRDCGSDDU,0.0,114708.0,12.0,2.0,7686.0,3.0,1.0,1.0,1.0
2,JDVF4U8JUANEID68,0.0,57329.0,6.0,4.0,2484.0,3.0,0.0,4.0,1.0
3,8I6SQDGP9OQYUN1M,0.0,28659.0,3.0,1.0,7754.0,6.0,0.0,1.0,1.0
4,R-TRDUV3GHTID31I,0.0,28674.0,3.0,4.0,4062.0,2.0,1.0,1.0,1.0


In [322]:
del_col = ['CUST_NO', 'VISITDATE', 'CUST_START_DT',]
scale_col = []

# Delete feature
for col in del_col:
    del train_X[col]
    del test_X[col]

# Standardizing
#from sklearn import preprocessing

#for col in scale_col:
#    train_X[col] = preprocessing.scale(train_X[col])
#    test_X[col] = preprocessing.scale(test_X[col])
train_X.head()

,count_x,Browse_count,AGE,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,2.0,3.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,4.0,6.0,0.0,1.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [323]:
print("The percentage of 1 label: ", train_y.sum()/train_y.count())

The percentage of 1 label:  label    0.044282
dtype: float64


In [324]:
filter_col = ['AGE', 'WORK_MTHS', 'EDU_CODE', 'Browse_count']

for col in filter_col:
    true_label = train_y['label'].values > 0
    other = train_X[col].values>0
    filt = true_label + other

    train_X = train_X[filt]
    train_y = train_y[filt]
    print('Filter=', col,'  ',train_X.shape, ' | ',train_y.shape)

Filter= AGE    (577302, 7)  |  (577302, 1)
Filter= WORK_MTHS    (470152, 7)  |  (470152, 1)
Filter= EDU_CODE    (469937, 7)  |  (469937, 1)
Filter= Browse_count    (118089, 7)  |  (118089, 1)


In [326]:
print("The percentage of 1 label: ", train_y.sum()/train_y.count())

The percentage of 1 label:  label    0.280788
dtype: float64


In [327]:
# Split training & validating set
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_X.values, train_y.values, test_size=0.2)

In [328]:
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(94471, 7) (23618, 7) (94471, 1) (23618, 1)


In [329]:
y_train = y_train.reshape((-1,))
y_valid = y_valid.reshape((-1,))

In [330]:
def evalute(pred, y_valid):
    return np.sum(pred == y_valid) / pred.shape[0]

# Random forest

In [331]:
from sklearn.ensemble import RandomForestClassifier

In [332]:
clf = RandomForestClassifier(n_estimators=5000, max_depth=5,
                              random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [333]:
eval_result = clf.predict(X_valid)
evalute(eval_result, y_valid)

0.9771784232365145

In [335]:
pred = clf.predict(test_X.values)
sub_df = pd.DataFrame()
sub_df['CUST_NO']=pd.read_csv("./TBN_Y_ZERO.csv")['CUST_NO']
sub_df['CC_IND']= pred
sub_df.to_csv('./pred/submi_CC.csv', index=False)

In [338]:
sub_df['CC_IND'].sum() / sub_df['CC_IND'].count()

0.1968